In [ ]:
import ee

# Initialize the Earth Engine module.
ee.Initialize()

# Import the FeatureCollection
fc = ee.FeatureCollection("projects/mapbiomas-workspace/SOLOS/AMOSTRAS/MATRIZES/soil_organic_carbon/matriz-collection1-amazonia-v3")

# Sort and print the column names
columns = fc.first().propertyNames().sort()
print(columns.getInfo())

In [ ]:
# Create a column called 'target'
# If the column id starts with ctb0053, ctb0055, ctb0056, ctb0057, ctb0058, ctb0059, ctb0060,
# or ctb0061, set the value of 'target' to 0. Otherwise, set the value of 'target' to 1.
def set_target(feature):
    id = feature.get('dataset_id')
    target = ee.Algorithms.If(
        ee.String(id).match('^ctb0053|^ctb0055|^ctb0056|^ctb0057|^ctb0058|^ctb0059|^ctb006[0-1]'),
        0,
        1
    )
    return feature.set('target', target)

# Map the function over the FeatureCollection
fc = fc.map(set_target)

# Print the first ten features
print(fc.limit(10).getInfo())

In [ ]:
# Print a summary of the 'target' column
print(fc.aggregate_histogram('target').getInfo())

In [ ]:
# Save the FeatureCollection to a new asset
output_asset_id = 'projects/mapbiomas-workspace/SOLOS/AMOSTRAS/MATRIZES/soil_organic_carbon/matriz-collection1-amazonia-v3-target'
task = ee.batch.Export.table.toAsset(collection=fc, description='export_fc', assetId=output_asset_id)
task.start()

In [ ]:
# In the fc FeatureCollection, filter the features with target equal to 0
fc_0 = fc.filter(ee.Filter.eq('target', 0))

# Count the unique values of the 'year' property
years = fc_0.aggregate_histogram('year')
print(years.getInfo())